In [ ]:
using Revise

In [ ]:
using Unitful
using AxisArrays
using Images: imadjustintensity
using Interpolations
using Plots
using StaticArrays: SVector
using VideoIO: openvideo

import CornerCameras
const CC = CornerCameras

In [ ]:
video = openvideo("example_videos/stereo/red_person_walking_circles.MP4")
background_im = CC.background(video, (10u"s", 20u"s"), 10)

In [ ]:
time_range = (0u"s", 30u"s")
reconstruction_framerate = 10/u"s"
λ = sqrt(2.7)
params = CC.Params();

In [ ]:
corners1 = [[131, 282], 
    [210, 979],
    [855, 784], 
    [680, 4], 
]
source1 = CC.StaticSource(video, corners1, background_im, λ)

In [ ]:
cam1 = CC.CornerCamera(source1, params)

In [ ]:
trace1 = CC.reconstruct(cam1, time_range, reconstruction_framerate)
imadjustintensity(trace1)

In [ ]:
path1 = [
    [114, 1],
    [146, 13],
    [14, 30],
    [123, 67],
    [124, 78],
    [25, 99],
    [127, 144],
    [126, 156],
    [27, 183],
    [124, 224],
    [124, 239],
    [30, 269],
    [89, 299]
    ]

path1 = extrapolate(interpolate(
    (last.(path1),),
    first.(path1),
        Gridded(Linear())),
    Flat()
    );

im = imadjustintensity(trace1)
for i in 1:size(trace1, 2)
    im[round(Int, path1[i]), i] = RGB(1., 1, 1)
end
im

In [ ]:
corners2 = [
    [477, 1522],
    [794, 1483],
    [856, 1873],
    [521, 1875]
]
source2 = CC.StaticSource(video, corners2, background_im, λ)

In [ ]:
cam2 = CC.CornerCamera(source2, params)

In [ ]:
trace2 = CC.reconstruct(cam2, time_range, reconstruction_framerate)
imadjustintensity(trace2)

In [ ]:
path2 = [
    [63, 4],
    [40, 14],
    [107, 34],
    [71, 53],
    [55, 67],
    [50, 81],
    [77, 96],
    [93, 100],
    [97, 110],
    [54, 144],
    [54, 163],
    [102, 185],
    [53, 227],
    [53, 244],
    [102, 273],
    [71, 298]
    ]

path2 = extrapolate(interpolate(
    (last.(path2),),
    first.(path2),
        Gridded(Linear())),
    Flat()
    );


im = imadjustintensity(trace2)
for i in 1:size(trace2, 2)
    im[round(Int, path2[i]), i] = RGB(1., 0, 0)
end
im

In [ ]:
B = 1.0
positions = map(1:size(trace1, 2)) do i
    θ1 = π/2 * path1[i] / size(trace1, 1) - π/2
    θ2 = π/2 * path2[i] / size(trace2, 1)
    Pz = B / (cot(θ1) + cot(θ2))
    Px = Pz * cot(θ1)
    SVector(Px, Pz)
end;

In [ ]:
plt = plot(first.(positions), 
           last.(positions), 
           linewidth=3, 
           color=1:length(positions),
           legend=false,
           xlabel="X-position",
           ylabel="Z-position",
           title="Inferred Position over Time (Scene B)")